# Executor: Serverless Code Execution with Azure Container Apps and Managed Code interpreter Sessions 

This notebook demonstrates how to:

- Initialize an **Azure Container Apps Session Pool** for our fully managed code interpreter sessions.
- Ensure the session pool exists (or create it if needed).
- Use the session pool to execute **Python code** in an **isolated, secure environment**.
- Handle authentication and role assignments automatically.

### **Key Features**
- Fully **serverless** Python execution environment.  
- Supports **synchronouss** code execution (for now).

## Install Required Libraries
Before starting, ensure the required libraries are installed:

In [ ]:
!pip install floki-ai python-dotenv azure-identity azure-mgmt-resource azure-mgmt-authorization nest_asyncio

## Load Environment Variables

We use `dotenv` to load configuration values from a `.env` file.  
Ensure your `.env` file contains: `AZURE_SUBSCRIPTION_ID=your_subscription_id`

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

AZURE_SUBSCRIPTION_ID = os.getenv("AZURE_SUBSCRIPTION_ID")
assert AZURE_SUBSCRIPTION_ID, "AZURE_SUBSCRIPTION_ID is not set in environment variables"

## Initialize Logging

In [2]:
import logging

logging.basicConfig(level=logging.INFO)
logging.getLogger("azure").setLevel(logging.WARNING)

## Initialize Azure Container Apps Session Pool

We will now create (or retrieve) an **Azure Container Apps Session Pool**.

In [3]:
from floki.executors import AzContainerAppsSessionPools

# Initialize session manager
session_manager = AzContainerAppsSessionPools(
    subscription_id=AZURE_SUBSCRIPTION_ID,
    resource_group="floki-sessionpool",
    session_pool_name="floki-session-pool",
    location="westus2"
)

# Retrieve or create the session pool
session_pool = session_manager.get_or_create_session_pool()

INFO:floki.executors.acasessionpools:Current registration state of 'Microsoft.App': registered
INFO:floki.executors.acasessionpools:Checking if resource group 'floki-sessionpool' exists...
INFO:floki.executors.acasessionpools:Resource group 'floki-sessionpool' already exists.
INFO:floki.executors.acasessionpools:Checking session pool 'floki-session-pool' in resource group 'floki-sessionpool'...
INFO:floki.executors.acasessionpools:Session pool 'floki-session-pool' already exists.
INFO:floki.executors.acasessionpools:Checking if user Roberto Rodriguez already has the required role...
INFO:floki.executors.acasessionpools:User Roberto Rodriguez already has the required role.


## Execute Code Using the Session Pool

Now that we have a session pool, we can execute Python code within it.

In [ ]:
from floki.executors import AzContainerAppsCodeExecutor

# Retrieve the session pool management endpoint
pool_management_endpoint = session_pool.pool_management_endpoint

# Initialize the Azure Container Code Executor
az_container_code_executor = AzContainerAppsCodeExecutor(
    pool_management_endpoint=pool_management_endpoint,
    api_version="2024-10-02-preview"
)

INFO:floki.executors.acacontainerapps:Using DefaultAzureCredential for authentication.
INFO:floki.executors.acacontainerapps:Using API version: 2024-10-02-preview


Let's define a simple Python snippet and execute it in the cloud.

In [5]:
from floki.types.executor import ExecutionRequest, CodeSnippet
import nest_asyncio
nest_asyncio.apply()

# Define a simple Python code snippet
execution_request = ExecutionRequest(
    snippets=[CodeSnippet(language="python", code="print('Hello from Azure Container Apps')")]
)

# Execute the code asynchronously
await az_container_code_executor.execute(execution_request)

[ExecutionResult(status='success', output='Hello from Azure Container Apps', exit_code=0)]